In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli
import tigre

In [3]:
# Load the mat files and extract the data
import scipy.io
import os

# Specify the directory where the mat files are located
directory = '/media/jericho/T7/Jericho_MAR_Studies/'

# Get the list of all the mat files in the directory
files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.mat')]
files.sort()

# Load the mat files into a pandas dataframe
data = []
for file in files:
    data.append(scipy.io.loadmat(file))


In [4]:

# Extract the data from the mat files into a pandas dataframe
# dict_keys(['__header__', '__version__', '__globals__', 'L', 'metadata'])
data_all = {}
for i in range(len(data)):
    f_name = files[i].split('/')[-1]
    data_all[f_name] = {}
    data_all[f_name]['projections'] = data[i]['L']
    data_all[f_name]['metadata'] = data[i]['metadata']


In [5]:
first_layer_bilat_proj = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L1 140kVp 25mAs.mat']['projections']
first_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L1 140kVp 25mAs.mat']['metadata']
secon_layer_bilat_proj = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L2 140kVp 25mAs.mat']['projections']
secon_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L2 140kVp 25mAs.mat']['metadata']

In [ ]:
first_layer_bilat_proj_nm = data_all['2024_01_20 CIRS801PF Spine NoMetal kV-CBCT DLI L1 140kVp 25mAs.mat']['projections']
first_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L1 140kVp 25mAs.mat']['metadata']
secon_layer_bilat_proj_nm = data_all['2024_01_20 CIRS801PF Spine NoMetal kV-CBCT DLI L1 140kVp 25mAs.mat']['projections']
secon_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L2 140kVp 25mAs.mat']['metadata']
first_layer_bilat_proj = data_all['2024_01_20 CIRS801PF Spine Metal_DoubleScrew kV-CBCT DLI L1 140kVp 25mAs.mat']['projections']
first_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L1 140kVp 25mAs.mat']['metadata']
secon_layer_bilat_proj = data_all['2024_01_20 CIRS801PF Spine Metal_DoubleScrew kV-CBCT DLI L2 140kVp 25mAs.mat']['projections']
secon_layer_bilat_header = data_all['2024_01_20 CIRS801PF Pelvis BiLat_Ti kV-CBCT DLI L2 140kVp 25mAs.mat']['metadata']

In [88]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter, disk
from skimage.util import img_as_ubyte
from scipy.interpolate import griddata


# Load picture and detect edges
scaled = first_layer_bilat_proj.T[400].copy()
image0 = scaled.copy()
scaled[scaled < 6] = 6
scaled -= np.min(scaled)
# scaled = first_layer_bilat_proj.T[0] - np.min(first_layer_bilat_proj.T[0])
scaled = scaled / np.max(scaled)
image = img_as_ubyte(scaled)
edges = canny(image, sigma=3, low_threshold=10, high_threshold=50)


# Detect two radii
hough_radii = np.arange(20, 45, 2)
hough_res = hough_circle(edges, hough_radii)

# Select the most prominent 3 circles
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                           total_num_peaks=2)

# Draw them
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
image = color.gray2rgb(image)
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = circle_perimeter(center_y, center_x, radius,
                                    shape=image.shape)
    image[circy, circx] = (220, 20, 20)

for center_y, center_x, radius in zip(cy, cx, radii):
    rr, cc = disk((center_y, center_x), radius+2)
    # Create a mask that only includes the coordinates within the image dimensions
    # mask = (rr >= 0) & (rr < image.shape[0]) & (cc >= 0) & (cc < image.shape[1])
    # rr, cc = rr[mask], cc[mask]
   
    # # Get the pixel values at the boundary of the circle
    # boundary_values = image0[rr, cc]
    # # Invert the boundary_values to get the values inside the circle
    # boundary_values = 1 - boundary_values
    rr[rr >= image0.shape[0]] = image0.shape[0] - 1
    cc[cc >= image0.shape[1]] = image0.shape[1] - 1

    image0[rr,cc] = np.NaN
    image0[image0 > 6.5] = np.NaN
    # # Create a grid of the same shape as the image
    # grid_y, grid_x = np.mgrid[0:image0.shape[0], 0:image0.shape[1]]
    
    # # Perform interpolation
    # interpolated_values = griddata((grid_y, grid_x), image0, (rr, cc), method='cubic')
    
    # # Replace the pixel values inside the circle with the interpolated values
    # image0[center_y-radius:center_y+radius, center_x-radius:center_x+radius] = interpolated_values[center_y-radius:center_y+radius, center_x-radius:center_x+radius]
    # break

ax.imshow(image, cmap=plt.cm.gray)
plt.show()

plt.figure()
plt.imshow(np.isnan(image0))
import numpy as np
import matplotlib.pyplot as plt

from skimage import data
from skimage.morphology import disk, binary_dilation
from skimage.restoration import inpaint

image_orig = data.astronaut()


image_defect = image0.copy()
mask =  np.isnan(image_defect)
image_result = inpaint.inpaint_biharmonic(image_defect, mask)

fig, axes = plt.subplots(ncols=2, nrows=2)
ax = axes.ravel()

ax[0].set_title('Original image')
ax[0].imshow(image_orig)

ax[1].set_title('Mask')
ax[1].imshow(mask, cmap=plt.cm.gray)

ax[2].set_title('Defected image')
ax[2].imshow(image_defect)

ax[3].set_title('Inpainted image')
ax[3].imshow(image_result)

for a in ax:
    a.axis('off')

fig.tight_layout()
plt.show()

In [7]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter, disk
from skimage.util import img_as_ubyte
from skimage.restoration import inpaint
from skimage.morphology import binary_dilation

def inpaint_images(images):
    results = []
    images2 = images.copy()
    for image0 in images2:
        # Process image
        scaled = image0.copy()
        scaled[scaled < 6] = 6
        scaled -= np.min(scaled)
        scaled = scaled / np.max(scaled)
        image = img_as_ubyte(scaled)
        edges = canny(image, sigma=3, low_threshold=10, high_threshold=50)

        # Detect two radii
        hough_radii = np.arange(20, 45, 2)
        # hough_res = hough_circle(edges, hough_radii)

        # Select the most prominent 3 circles
        # accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=2)

        # for center_y, center_x, radius in zip(cy, cx, radii):
        #     rr, cc = disk((center_y, center_x), radius+8)
        #     rr[rr >= image0.shape[0]] = image0.shape[0] - 1
        #     cc[cc >= image0.shape[1]] = image0.shape[1] - 1

        #     image0[rr,cc] = np.NaN
        
        perc_90 = np.percentile(image0, 92)
        if perc_90 > 6.9:
            image0[image0 > perc_90] = np.NaN
        else:
            image0[image0 > 7.2] = np.NaN

        # Make the values above below and to the side of the NaNs NaN
        mask = np.isnan(image0)
        expanded_mask = binary_dilation(mask)
        expanded_mask = binary_dilation(expanded_mask)
        image0[expanded_mask] = np.NaN

        # for center_y, center_x, radius in zip(cy, cx, radii):
        #     circy, circx = circle_perimeter(center_y, center_x, radius+8,
        #                                     shape=image.shape)
        #     image0[circy, circx] = 10
        #     perc_90 = np.percentile(image0, 95)
        #     if perc_90 > 7.2:
        #         image0[image0 > perc_90] = 10
        #     else:
        #         image0[image0 > 7.2] = 10
            # image0[np.image0 > 7.2] = 10
        # # Inpaint image using linear interpolation
        # x = np.arange(0, image0.shape[1])
        # y = np.arange(0, image0.shape[0])
        # array = np.ma.masked_invalid(image0)
        # xx, yy = np.meshgrid(x, y)
        # x1 = xx[~array.mask]
        # y1 = yy[~array.mask]
        # newarr = array[~array.mask]

        # image_result = griddata((x1, y1), newarr.ravel(), (xx, yy), method='linear')
        # results.append(image_result)
        # Inpaint image
        mask =  np.isnan(image0)
        image_result = inpaint.inpaint_biharmonic(image0, mask)
        results.append(image_result)
        # results.append(image0)
    return results

def inpaint_images_spine(images0,images1):
    results = []
    images2 = images0.copy()
    images3 = images1.copy()

    p_98 = 7.1 #np.percentile(images2, 98)
    
    for image_high, image_low in zip(images2, images3):
        # Process image
        # scaled = image0.copy()
        # scaled[scaled < 4] = 4
        # scaled -= np.min(scaled)
        # scaled = scaled / np.max(scaled)
        # image = img_as_ubyte(scaled)
        # edges = canny(image, sigma=3, low_threshold=10, high_threshold=50)

        # # Detect two radii
        # hough_radii = np.arange(20, 45, 2)

        # perc_90 = np.percentile(image0, 92)
        # if perc_90 > 6.9:
        p_98 = np.percentile(image_high, 99.5)
        # p_80 = np.percentile(image_high, 80)

        ratio0 = image_high/(image_low)

        mask = np.zeros_like(image_high)
        mask[240:,125:145 ] = ratio0[240:,125:145 ] > 1.025
        mask[240:,125:145 ] += image_high[240:,125:145 ] > p_98
        mask[240:290,125:145 ] += ratio0[240:290,125:145 ]*image_high[240:290,125:145 ] > 6.5*1.1
        mask[mask == 2] = 1
        mask[mask == 3] = 1
        image_high[mask.astype(bool)] = np.NaN
        # image_high[image_high > p_98] = np.NaN
        # Make the values above below and to the side of the NaNs NaN
        # mask = np.isnan(image_high)
        expanded_mask = binary_dilation(mask)
        expanded_mask = binary_dilation(expanded_mask)
        expanded_mask = binary_dilation(expanded_mask)
        expanded_mask = binary_dilation(expanded_mask)
        expanded_mask = binary_dilation(expanded_mask)

        image_high[expanded_mask] = np.NaN
        # Inpaint image
        mask =  np.isnan(image_high)
        image_result = inpaint.inpaint_biharmonic(image_high, mask)
        results.append(image_result)
        # results.append(image_high)
    return results
# Inpaint the images
# inpaint_images_LI = inpaint_images(first_layer_bilat_proj)
# image0 = first_layer_bilat_proj.copy()
# image1 = secon_layer_bilat_proj.copy()

# image0[image0 < 4.5] = 0
# image1[image1 < 4.5] = 1

# ratio = image0/(image1)

images_LI = inpaint_images_spine(first_layer_bilat_proj.T[:],secon_layer_bilat_proj.T[:])

/tmp/ipykernel_36703/3836162137.py:102: RuntimeWarning: divide by zero encountered in divide
  ratio0 = image_high/(image_low)
/tmp/ipykernel_36703/3836162137.py:102: RuntimeWarning: invalid value encountered in divide
  ratio0 = image_high/(image_low)


In [131]:
plt.figure()
pli.hyperslicer(np.array(images_LI),vmin=0,vmax=7.1)

AttributeError: 'Controls' object has no attribute 'vbox'

In [8]:
# Plot the inpainted images
image0 = first_layer_bilat_proj.copy()
image1 = secon_layer_bilat_proj.copy()

image0[image0 < 1] = 0
image1[image1 < 1] = 1

ratio = image0/(image1)

plt.figure()
plt.subplot(1,2,1)
cc = pli.hyperslicer(ratio.T,vmin=0.9,vmax=1.3)
plt.subplot(1,2,2)
c2 = pli.hyperslicer(image0.T,controls=cc)

libGL error: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: radeonsi
libGL error: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: radeonsi
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [12]:
import mpl_interactions.ipyplot as iplt

plt.figure()
iplt.imshow((first_layer_bilat_proj.T/(secon_layer_bilat_proj.T+0.0001))[645],vmin_vmax=(0.9,1.3))

AttributeError: 'Controls' object has no attribute 'vbox'

In [133]:
plt.figure()
plt.subplot(1,2,1)
cc = pli.hyperslicer(first_layer_bilat_proj.T, autoscale_cmap=False)
plt.subplot(1,2,2)
c2 = pli.hyperslicer(secon_layer_bilat_proj.T, autoscale_cmap=False,controls=cc)

In [18]:
np.percentile(image0, 98)

6.753761625289918

In [9]:
geo = tigre.geometry_default()

In [10]:
plt.figure()
plt.plot(secon_layer_bilat_header[0][-1][2][1,:], secon_layer_bilat_header[0][-1][2][0,:], 'o')
plt.plot(first_layer_bilat_header[0][-1][1][1,:], first_layer_bilat_header[0][-1][1][0,:], 'o')

source_pos_x = secon_layer_bilat_header[0][-1][2][1,:]
source_pos_y = secon_layer_bilat_header[0][-1][2][0,:]
detector_pos_x = first_layer_bilat_header[0][-1][1][1,:]
detector_pos_y = first_layer_bilat_header[0][-1][1][0,:]

# calculate the angles from the source to the detector
angles_real = np.arctan2(detector_pos_x - source_pos_x, detector_pos_y - source_pos_y)

# calculate the distance from the source to the detector
distance = np.sqrt((detector_pos_x - source_pos_x)**2 + (detector_pos_y - source_pos_y)**2)

#plot the angles and the distance
plt.figure()
plt.plot(angles_real, 'o')

plt.figure()
plt.plot(distance)
# print(np.mean(secon_layer_bilat_header[0][-1][2][1,:]), np.mean(secon_layer_bilat_header[0][-1][2][0,:]), 'o')

In [11]:
import h5py
import numpy as np

class Geo:
    pass

def assign_values_to_geo(filename):
    geo = Geo()
    with h5py.File(filename, 'r') as f:
        def assign_value(name, obj):
            if isinstance(obj, h5py.Dataset):
                # Remove 'geo/' from the name and replace any remaining '/' with '_'
                attr_name = name.replace('geo/', '').replace('/', '_')
                setattr(geo, attr_name, np.array(obj[0][()]))
        f.visititems(assign_value)
    return geo

# Use the function
geo2 = assign_values_to_geo('/home/jericho/Downloads/firefox-downloads/geo.mat')

# Now you can access the values like this:
print(geo2.DSD)
print(geo2.offDetector)
# print_hdf5_file_structure()

[1500.]
[160.   0.]


In [12]:
geo.DSD = 1527.3
geo.DSO = 1000
# geo.mode = geo2.mode
geo.nDetector = np.array([266, 354])
geo.dDetector = np.array([1.12,1.12])
geo.sDetector = geo.nDetector*geo.dDetector
geo.offDetector = np.array([0,-160])# geo2.offDetector
geo.nVoxel = np.array([120,512,512])
geo.dVoxel = np.array([1.2,1,1])
geo.sVoxel = geo.nVoxel*geo.dVoxel
geo.offOrigin = geo2.offOrigin

In [13]:
angles = angles_real 
# Shift the angles by 110 degrees 
angles = angles + np.deg2rad(-96)
# Make sure that the angles are between 0 and 2*pi
angles = np.mod(angles, 2*np.pi)


In [14]:
# print(np.array(images_LI).transpose([0,2,1]).shape,first_layer_bilat_proj.transpose([2,0,1]).shape)
img_LI = tigre.algorithms.fdk(np.array(images_LI).transpose([0,2,1]),geo,angles,dowang=True)

FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
Detector parameters
Number of pixels (nDetector) = [266 639]
Size of each pixel (dDetector) = [1.12 1.12] mm
Total size of the detector (sDetector) = [297.92 715.68] mm
-----
Image parameters
Number of voxels (nVoxel) = [120 512 512]
Total size of the image (sVoxel) = [144. 512. 512.] mm
Size of each voxel (dVoxel) = [1.2 1.  1. ] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0. 0. 0.] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
(894, 266, 639)


In [31]:
img_first = tigre.algorithms.fdk(first_layer_bilat_proj.transpose([2,0,1]),geo,angles,dowang=True)
img_second = tigre.algorithms.fdk(secon_layer_bilat_proj.transpose([2,0,1]),geo,angles,dowang=True)
# img_combined = tigre.algorithms.fdk(np.array(images_LI).transpose([0,2,1]),geo,angles,dowang=True)

FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
Detector parameters
Number of pixels (nDetector) = [266 639]
Size of each pixel (dDetector) = [1.12 1.12] mm
Total size of the detector (sDetector) = [297.92 715.68] mm
-----
Image parameters
Number of voxels (nVoxel) = [120 512 512]
Total size of the image (sVoxel) = [144. 512. 512.] mm
Size of each voxel (dVoxel) = [1.2 1.  1. ] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0. 0. 0.] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
(894, 266, 639)
FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
D

In [15]:
plt.figure()
plt.subplot(1,3,1)
plt.imshow

In [16]:
img = tigre.algorithms.fdk(first_layer_bilat_proj.transpose([2,0,1]), geo, angles,dowang=True)

FDK: applying detector offset weights
-285 639 0 -38.24000000000001
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1527.3 mm
Distance from source to origin (DSO)= 1000 mm
-----
Detector parameters
Number of pixels (nDetector) = [266 639]
Size of each pixel (dDetector) = [1.12 1.12] mm
Total size of the detector (sDetector) = [297.92 715.68] mm
-----
Image parameters
Number of voxels (nVoxel) = [120 512 512]
Total size of the image (sVoxel) = [144. 512. 512.] mm
Size of each voxel (dVoxel) = [1.2 1.  1. ] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0. 0. 0.] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
(894, 266, 639)


In [32]:
def mask_center(image, radius):
    # Get the center of the image
    center = np.array(image.shape) / 2

    # Create an array of the same shape as the image, filled with the squared distance from the center
    y, x = np.ogrid[:image.shape[0], :image.shape[1]]
    dist_from_center = (x - center[1])**2 + (y - center[0])**2

    # Create a mask that is True for all pixels within the specified radius of the center
    mask = dist_from_center <= radius**2

    # Use the mask to set the corresponding pixels in the original image to 0
    image[~mask] = 0

    return image

# crop the image from 512 to 416
img = img[:,48:464, 48:464]
# Test the function
image = np.ones((100, 100))
radius = 416//2
for im in img:
    im = mask_center(im, radius)

img_LI = img_LI[:,48:464, 48:464]
for im in img_LI:
    im = mask_center(im, radius)

img[img < 0.01] = 0
img_LI[img_LI < 0.01] = 0

# do the same for the first and second layers

img_first = img_first[:,48:464, 48:464]
for im in img_first:
    im = mask_center(im, radius)

img_second = img_second[:,48:464, 48:464]
for im in img_second:
    im = mask_center(im, radius)

img_first[img_first < 0.01] = 0
img_second[img_second < 0.01] = 0

plt.figure()
plt.subplot(1,2,1)
plt.imshow(img_first[62])
plt.subplot(1,2,2)
plt.imshow(img_second[62])

In [22]:


def mask_center(image, radius):
    # Get the center of the image
    center = np.array(image.shape) / 2

    # Create an array of the same shape as the image, filled with the squared distance from the center
    y, x = np.ogrid[:image.shape[0], :image.shape[1]]
    dist_from_center = (x - center[1])**2 + (y - center[0])**2

    # Create a mask that is True for all pixels within the specified radius of the center
    mask = dist_from_center <= radius**2

    # Use the mask to set the corresponding pixels in the original image to 0
    image[~mask] = 0

    return image

# crop the image from 512 to 416
img = img[:,48:464, 48:464]
# Test the function
image = np.ones((100, 100))
radius = 416//2
for im in img:
    im = mask_center(im, radius)

img_LI = img_LI[:,48:464, 48:464]
for im in img_LI:
    im = mask_center(im, radius)

img[img < 0.01] = 0
img_LI[img_LI < 0.01] = 0
# plt.figure()
# pli.hyperslicer(img)

In [70]:
img.shape

(120, 512, 512)

In [18]:
# load an h5py  file
import h5py
import numpy as np

def print_hdf5_file_structure(file_name):
    def print_attrs(name, obj):
        print(name)
        for key, val in obj.attrs.items():
            print("    %s: %s" % (key, val))

    with h5py.File(file_name, 'r') as f:
        f.visititems(print_attrs)

fname = '/home/jericho/Software/dl_mar/OSCNet/data/test/test_640geo/000376_02_01/040/0.h5'

print_hdf5_file_structure(fname)

LI_CT
LI_sinogram
ma_CT
ma_sinogram
metal_trace


In [28]:
def test_image(data_path, imag_idx, mask_idx):
    txtdir = os.path.join(data_path, 'test_640geo_dir.txt')
    mat_files = open(txtdir, 'r').readlines()
    gt_dir = mat_files[imag_idx]
    file_dir = gt_dir[:-6]
    data_file = file_dir + str(mask_idx) + '.h5'
    abs_dir = os.path.join(data_path, 'test_640geo/', data_file)
    gt_absdir = os.path.join(data_path, 'test_640geo/', gt_dir[:-1])
    gt_file = h5py.File(gt_absdir, 'r')
    Xgt = gt_file['image'][()]
    gt_file.close()
    file = h5py.File(abs_dir, 'r')
    Xma= file['ma_CT'][()]
    XLI =file['LI_CT'][()]
    file.close()

    return Xgt, Xma, XLI
    # M512 = test_mask[:,:,mask_idx]
data_path = '/home/jericho/Software/dl_mar/OSCNet/data/test'
Xgt, Xma, XLI = test_image(data_path, 0, 0)


In [30]:
plt.figure()
pli.hyperslicer([Xgt,Xma,XLI])

AttributeError: 'Controls' object has no attribute 'vbox'

In [26]:
plt.figure()
plt.hist(Xma.flatten(), histtype='step',bins=100, alpha=0.5, label='ma_ct',log=True)
plt.hist(XLI.flatten(), histtype='step',bins=100, alpha=0.5, label='li_ct',log=True)
plt.hist(img[62].flatten(), histtype='step',bins=100, alpha=0.5, label='fdk_ma_ct',log=True)
plt.hist(img_LI[62].flatten(), histtype='step',bins=100, alpha=0.5, label='fdk_li_ct',log=True)
plt.legend()

In [27]:
# Open 0.h5 replace Xma with the img[55] and save as h5py file 1.h5
import h5py

fname2 = '/home/jericho/Software/dl_mar/OSCNet/data/test/test_640geo/000376_02_01/040/0.h5'

# Load the file
import h5py
import numpy as np

# Open the file in 'r+' mode
with h5py.File(fname2, 'r+') as f:
    # Load the images 
    f['ma_CT'][()] = img[62]
    f['LI_CT'][()] = img_LI[62]
    # The changes are automatically saved when the file is closed

In [94]:
%matplotlib qt

#%% DEMO 14:  Playing with offsets
#
#
# In this demo we show how to change offsets to either the iamge or the
# detector, and the flexibility of it.
#
#
# --------------------------------------------------------------------------
# --------------------------------------------------------------------------
# This file is part of the TIGRE Toolbox
#
# Copyright (c) 2015, University of Bath and
#                     CERN-European Organization for Nuclear Research
#                     All rights reserved.
#
# License:            Open Source under BSD.
#                     See the full license at
#                     https://github.com/CERN/TIGRE/blob/master/LICENSE
#
# Contact:            tigre.toolbox@gmail.com
# Codes:              https://github.com/CERN/TIGRE/
# Coded by:           Ander Biguri
# --------------------------------------------------------------------------
## Initialize
#%%Initialize
import tigre
import numpy as np
from tigre.utilities import sample_loader
import matplotlib.pyplot as plt
import tigre.algorithms as algs

#%% Geometry
geo = tigre.geometry_default(high_resolution=False)


# Offsets
## ####################################################################################
# Lets try simnple offset: The detector gets completelly displaced
geo.offOrigin = np.array([0, 0, 0])  # Offset of image from origin   (mm)
geo.offDetector = np.array([200, 200])  # Offset of Detector            (mm)
## ####################################################################################
# Auxiliary
geo.accuracy = 0.5
# Accuracy of FWD proj          (vx/sample)

## Load data and generate projections
# see previous demo for explanation
angles = np.linspace(0, 2 * np.pi, 100)

# Load thorax phatom data
head = sample_loader.load_head_phantom(geo.nVoxel)
projections = tigre.Ax(head, geo, angles)


#%% lets see it
# tigre.plotproj(projections, angles)

## we will skip reconstruction of this tests because the image is outside the detector


## #####################################################################

#%% Second test: lets test variying offsets:
geo.nDetector = np.array([80,120])  # Number of detector pixels
geo.dDetector = np.array([4, 4])  # Detector element size (mm)
geo.sDetector = geo.nDetector * geo.dDetector  # Detector size (mm)
geo.offDetector = np.array(
    [0,-160]
)  # Offset of Detector            (mm)
projections2 = tigre.Ax(head, geo, angles)
## lets see it
# tigre.plotproj(projections2, angles)
## reconstruction
geo.offDetector = np.array(
    [0,-160]
)  

res = algs.fdk(projections2, geo, angles, dowang=True)


# tigre.plotimg(res, dim="z")

#%% Third test: lets vary everything
# plt.show()

FDK: applying detector offset weights
-80 200 0 -80.0
TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1536.0 mm
Distance from source to origin (DSO)= 1000.0 mm
-----
Detector parameters
Number of pixels (nDetector) = [ 80 200]
Size of each pixel (dDetector) = [4 4] mm
Total size of the detector (sDetector) = [320 800] mm
-----
Image parameters
Number of voxels (nVoxel) = [64 64 64]
Total size of the image (sVoxel) = [256 256 256] mm
Size of each voxel (dVoxel) = [4. 4. 4.] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0 0 0] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5
-----
Rotation of the Detector (rotDetector) = [0 0 0] rad
(100, 80, 200)


In [86]:
# load a png file and plot it using imshow
png_file = '/home/jericho/Software/dl_mar/OSCNet/save_results/osc/1.png'
import mpl_interactions.ipyplot as iplt
img = plt.imread(png_file)
plt.figure()
cc = iplt.imshow(img.mean(axis=2),vmin_vmax=(0,1))

In [96]:
plt.figure()
pli.hyperslicer(res)

AttributeError: 'Controls' object has no attribute 'vbox'

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import tigre

In [8]:
import mpl_interactions as pli


plt.figure()
# pli.hyperslicer(first_layer_bilat_proj.transpose([2,0,1]))
pli.hyperslicer(projections2)

AttributeError: 'Controls' object has no attribute 'vbox'